In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
%matplotlib inline

In [2]:
import keras

Using TensorFlow backend.


In [3]:
model_tmp = keras.models.load_model('model_audio.h5')
model_audio = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [4]:
model_tmp = keras.models.load_model('model_eyes.h5')
model_eyes = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [5]:
model_tmp = keras.models.load_model('model_face.h5')
model_face = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [6]:
model_tmp = keras.models.load_model('model_kinect.h5')
model_kinect = keras.models.Model(model_tmp.input, model_tmp.get_layer('dropout_2').output)

In [7]:
del model_tmp

In [8]:
def norm(arr, mins, maxs):
    arr = arr.astype(float)
    for i in range(arr.shape[-1]):
        arr[:, :, i] -= mins[i]
        arr[:, :, i] /= maxs[i]
    return arr

In [9]:
mins_audio, maxs_audio = np.load('audio_mins.npy'), np.load('audio_maxs.npy')
mins_kinect, maxs_kinect = np.load('kinect_mins.npy'), np.load('kinect_maxs.npy')

In [10]:
# X_train_audio = norm(np.load('X_train_audio.npy'), mins_audio, maxs_audio)
X_val_audio = norm(np.load('X_val_audio.npy'), mins_audio, maxs_audio)

# X_train_eyes = np.load('X_train_eyes.npy')
X_val_eyes = np.load('X_val_eyes.npy')

# X_train_face = np.load('X_train_face.npy')
X_val_face = np.load('X_val_face.npy')

# X_train_kinect = norm(np.load('X_train_kinect.npy'), mins_kinect, maxs_kinect)
X_val_kinect = norm(np.load('X_val_kinect.npy'), mins_kinect, maxs_kinect)

print X_val_audio.shape, X_val_eyes.shape, X_val_face.shape, X_val_kinect.shape

(2114, 400, 36) (2114, 200, 6) (2114, 200, 100) (2114, 60, 27)


In [13]:
y_val = np.load('y_val.npy')

print y_val.shape

(2114,)


In [11]:
nn_audio_val = model_audio.predict(X_val_audio)
nn_eyes_val = model_eyes.predict(X_val_eyes)
nn_face_val = model_face.predict(X_val_face)
nn_kinect_val = model_kinect.predict(X_val_kinect)

print nn_audio_val.shape, nn_eyes_val.shape, nn_face_val.shape, nn_kinect_val.shape

(2114, 30) (2114, 12) (2114, 40) (2114, 20)


In [19]:
predictor_audio = RandomForestClassifier(n_estimators=100)
predictor_eyes = RandomForestClassifier(n_estimators=100)
predictor_face = RandomForestClassifier(n_estimators=100)
predictor_kinect = RandomForestClassifier(n_estimators=100)

In [20]:
scores = cross_val_score(predictor_audio, nn_audio_val, y_val, cv=4)
print 'audio:', scores, np.mean(scores)
scores = cross_val_score(predictor_eyes, nn_eyes_val, y_val, cv=4)
print 'eyes:', scores, np.mean(scores)
scores = cross_val_score(predictor_face, nn_face_val, y_val, cv=4)
print 'face:', scores, np.mean(scores)
scores = cross_val_score(predictor_kinect, nn_kinect_val, y_val, cv=4)
print 'kinect:', scores, np.mean(scores)

audio: [ 0.44256121  0.40377358  0.38636364  0.42857143] 0.415317463778
eyes: [ 0.39171375  0.29433962  0.26325758  0.26095238] 0.302565831749
face: [ 0.58380414  0.45283019  0.60037879  0.6       ] 0.559253279921
kinect: [ 0.44256121  0.27735849  0.33712121  0.41904762] 0.369022131752


In [27]:
weak_audio_val = cross_val_predict(predictor_audio, nn_audio_val, y_val, cv=4, method='predict_proba')
weak_eyes_val = cross_val_predict(predictor_eyes, nn_eyes_val, y_val, cv=4, method='predict_proba')
weak_face_val = cross_val_predict(predictor_face, nn_face_val, y_val, cv=4, method='predict_proba')
weak_kinect_val = cross_val_predict(predictor_kinect, nn_kinect_val, y_val, cv=4, method='predict_proba')

print weak_audio_val.shape, weak_eyes_val.shape, weak_face_val.shape, weak_kinect_val.shape

(2114, 6) (2114, 6) (2114, 6) (2114, 6)


In [32]:
weights_weak = [0.42, 0.3, 0.56, 0.37]

# preds_weak = 0.42 * weak_audio_val + 0.3 * weak_eyes_val + 0.56 * weak_face_val + 0.37 * weak_kinect_val
preds_weak = weak_audio_val + weak_eyes_val + weak_face_val + weak_kinect_val

In [33]:
y_pred = np.argmax(preds_weak, axis=1)

print y_pred.shape

(2114,)


In [34]:
print np.mean(y_pred != y_val)

0.450331125828


In [35]:
nn_val = np.concatenate([nn_audio_val, nn_eyes_val, nn_face_val, nn_kinect_val], axis=1)

print nn_val.shape

(2114, 102)


In [38]:
predictor = RandomForestClassifier(n_estimators=1000)

scores = cross_val_score(predictor, nn_val, y_val, cv=3, verbose=2)
print scores, np.mean(scores)

[CV]  ................................................................
[CV] ................................................. , total=   6.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.3s remaining:    0.0s


[CV] ................................................. , total=   6.1s
[CV]  ................................................................
[CV] ................................................. , total=   6.1s
[ 0.61189802  0.56028369  0.66571835] 0.612633351623


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   18.4s finished
